<a href="https://colab.research.google.com/github/dannygallagher/CIS519-Project/blob/main/Baseline_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import gzip
import json
import json
import gzip
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import svm
def get_metrics(true_labels, pred_labels):
    f1 = f1_score(true_labels, pred_labels, average='macro')
    acc = accuracy_score(true_labels, pred_labels)
    rec = recall_score(true_labels, pred_labels, average='macro')
    prec = precision_score(true_labels, pred_labels, average='macro')
    return f1, acc, rec, prec

def fast_parse(path):
  # path = path + ".json.gz"
  max_reviews = 1689188
  iter = 5000
  counter = 0 
  all_reviews = []
  g = open(path, 'r')
  for line in g:
    d = json.loads(line)
    all_reviews.append(d)
    if (counter % iter) == 0:
      print(f"On line{counter}, {counter/max_reviews} % done")
    counter = counter + 1
  return all_reviews

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [ ]:
col_list = ['Type', "Model", "Train Set", 'Test Set', 'F1', 'Accuracy', 'Recall', 'Precision']
total_results = pd.DataFrame(columns=col_list)

All 5 Cat

In [ ]:
output_dir ='/content/drive/MyDrive/cis519_final/Baselines/Naive Bayes/results_All_5Cat.csv'
# df_train = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_train.json')
# df_test = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_test.json')
# df_experiment = pd.from_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/elec_test_5class.csv')
# df_train = pd.DataFrame().from_dict(df_train)
# df_test = pd.DataFrame().from_dict(df_test)

df_train = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_train.csv')
df_test = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_test.csv')
df_exp = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/electronics_test.csv')

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

token = RegexpTokenizer(r'[a-zA-Z0-9]+')

cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
tf_idf = TfidfTransformer()
train_doc_matrix = cv.fit_transform(df_train['title_plus_review'])
X_train = tf_idf.fit_transform(train_doc_matrix)

test_doc_matrix = cv.transform(df_test['title_plus_review'])
X_test = tf_idf.transform(test_doc_matrix)

exp_doc_matrix = cv.transform(df_exp['title_plus_review'])
X_exp = tf_idf.transform(exp_doc_matrix)


y_train = df_train['overall']
y_test = df_test['overall']
y_exp = df_exp['overall']

from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(text_counts, df['overall'], test_size = 0.25, random_state = 5)



clf = svm.SVC()
clf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
df_train.head()

,Unnamed: 0,reviewText,summary,overall,title_plus_review,pos_neg,neutrality,num_words,nw
0,0,"Excellent quality, will have to remember to em...",Great looking and high quality,5.0,great looking and high quality excellent quali...,1,1,31,26
1,1,Product arrived on time and fit well. The only...,Product arrived on time and fit well. The only...,3.0,product arrived on time and fit well . the on...,2,0,29,20
2,2,Probably won't watch it again.,Moment After,3.0,moment after probably will not watch it again .,2,0,8,6
3,3,"very nice, soft ,comfortable and very flatteri...",4 stars,4.0,"4 stars very nice , soft , comfortable and v...",1,2,28,26
4,4,My daughter received this as a birthday gift a...,Yucky Yucky Yucky,1.0,yucky yucky yucky my daughter received this as...,0,1,104,101


In [ ]:
results = pd.DataFrame(columns=col_list)
#train scores
train_predicted = clf.predict(X_train)
f1, acc, rec, prec = get_metrics(y_train, train_predicted)
results = results.append(pd.DataFrame([["5Cat", "Naive Bayes", "All", "All-Train", f1, acc, rec, prec]], columns =col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#test scores
test_predicted = clf.predict(X_test)
f1, acc, rec, prec = get_metrics(y_test, test_predicted)
results = results.append(pd.DataFrame([["5Cat", "Naive Bayes", "All", "All-Test", f1, acc, rec, prec]], columns=col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#Experiment Scores
exp_predicted = clf.predict(X_exp)
f1, acc, rec, prec = get_metrics(y_exp, exp_predicted)
results = results.append(pd.DataFrame([["5Cat", "Naive Bayes", "All", "Elec-Test", f1, acc, rec, prec]], columns=col_list))
print(str('{:04.2f}'.format(f1*100))+'%')

results.to_csv(output_dir, index=False)
total_results = total_results.append(results)

58.13%
40.49%
43.08%


In [ ]:
results

,Type,Model,Train Set,Test Set,F1,Accuracy,Recall,Precision
0,5Cat,Naive Bayes,All,All-Train,0.581275,0.583144,0.583196,0.581655
0,5Cat,Naive Bayes,All,All-Test,0.404942,0.529363,0.493736,0.385027
0,5Cat,Naive Bayes,All,Elec-Test,0.430789,0.450334,0.456689,0.454546


Elec 5 Cat

In [ ]:
output_dir ='/content/drive/MyDrive/cis519_final/Baselines/Naive Bayes/results_Elec_5Cat.csv'
# df_train = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_train.json')
# df_test = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_test.json')
# df_experiment = pd.from_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/elec_test_5class.csv')
# df_train = pd.DataFrame().from_dict(df_train)
# df_test = pd.DataFrame().from_dict(df_test)

df_train = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/electronics_train.csv')
df_test = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/electronics_test.csv')
df_exp = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_test.csv')

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
tf_idf = TfidfTransformer()
train_doc_matrix = cv.fit_transform(df_train['title_plus_review'])
X_train = tf_idf.fit_transform(train_doc_matrix)

test_doc_matrix = cv.transform(df_test['title_plus_review'])
X_test = tf_idf.transform(test_doc_matrix)

exp_doc_matrix = cv.transform(df_exp['title_plus_review'])
X_exp = tf_idf.transform(exp_doc_matrix)


y_train = df_train['overall']
y_test = df_test['overall']
y_exp = df_exp['overall']

from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(text_counts, df['overall'], test_size = 0.25, random_state = 5)



clf = svm.SVC()
clf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
col_list = ['Type', "Model", "Train Set", 'Test Set', 'F1', 'Accuracy', 'Recall', 'Precision']
results = pd.DataFrame(columns=col_list)
#train scores
train_predicted = clf.predict(X_train)
f1, acc, rec, prec = get_metrics(y_train, train_predicted)
results = results.append(pd.DataFrame([["5Cat", "Naive Bayes", "Elec", "Elec-Train", f1, acc, rec, prec]], columns =col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#test scores
test_predicted = clf.predict(X_test)
f1, acc, rec, prec = get_metrics(y_test, test_predicted)
results = results.append(pd.DataFrame([["5Cat", "Naive Bayes", "Elec", "Elec-Test", f1, acc, rec, prec]], columns=col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#Experiment Scores
exp_predicted = clf.predict(X_exp)
f1, acc, rec, prec = get_metrics(y_exp, exp_predicted)
results = results.append(pd.DataFrame([["5Cat", "Naive Bayes", "Elec", "ALL-Test", f1, acc, rec, prec]], columns=col_list))
print(str('{:04.2f}'.format(f1*100))+'%')

results.to_csv(output_dir, index=False)
total_results = total_results.append(results)


57.06%
49.51%
37.75%


In [ ]:
results

,Type,Model,Train Set,Test Set,F1,Accuracy,Recall,Precision
0,5Cat,Naive Bayes,Elec,Elec-Train,0.570596,0.571873,0.572109,0.570359
0,5Cat,Naive Bayes,Elec,Elec-Test,0.495058,0.503864,0.501400,0.496625
0,5Cat,Naive Bayes,Elec,ALL-Test,0.377504,0.511363,0.444680,0.364700


ALL Binary

In [ ]:
output_dir ='/content/drive/MyDrive/cis519_final/Baselines/Naive Bayes/results_All_bin.csv'
df_train = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_train_binaryposneg.json')
df_test = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_test_binaryposneg.json')
df_exp = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/electronics_test_binaryposneg.json')
df_train = pd.DataFrame().from_dict(df_train)
df_test = pd.DataFrame().from_dict(df_test)
df_exp = pd.DataFrame().from_dict(df_exp)

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
tf_idf = TfidfTransformer()
train_doc_matrix = cv.fit_transform(df_train['title_plus_review'])
X_train = tf_idf.fit_transform(train_doc_matrix)

test_doc_matrix = cv.transform(df_test['title_plus_review'])
X_test = tf_idf.transform(test_doc_matrix)

exp_doc_matrix = cv.transform(df_exp['title_plus_review'])
X_exp = tf_idf.transform(exp_doc_matrix)


y_train = df_train['pos_neg']
y_test = df_test['pos_neg']
y_exp = df_exp['pos_neg']

from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(text_counts, df['overall'], test_size = 0.25, random_state = 5)



clf = svm.SVC()
clf.fit(X_train, y_train)

On line0, 0.0 % done
On line5000, 0.002960002083841467 % done
On line10000, 0.005920004167682934 % done
On line15000, 0.008880006251524401 % done
On line20000, 0.011840008335365868 % done
On line25000, 0.014800010419207334 % done
On line30000, 0.017760012503048803 % done
On line35000, 0.02072001458689027 % done
On line40000, 0.023680016670731736 % done
On line45000, 0.026640018754573204 % done
On line50000, 0.02960002083841467 % done
On line55000, 0.03256002292225614 % done
On line60000, 0.035520025006097605 % done
On line65000, 0.03848002708993907 % done
On line70000, 0.04144002917378054 % done
On line75000, 0.04440003125762201 % done
On line80000, 0.04736003334146347 % done
On line85000, 0.050320035425304936 % done
On line90000, 0.05328003750914641 % done
On line95000, 0.05624003959298787 % done
On line100000, 0.05920004167682934 % done
On line105000, 0.06216004376067081 % done
On line110000, 0.06512004584451228 % done
On line115000, 0.06808004792835375 % done
On line120000, 0.071040

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
col_list = ['Type', "Model", "Train Set", 'Test Set', 'F1', 'Accuracy', 'Recall', 'Precision']
results = pd.DataFrame(columns=col_list)
#train scores
train_predicted = clf.predict(X_train)
f1, acc, rec, prec = get_metrics(y_train, train_predicted)
results = results.append(pd.DataFrame([["Binary", "Naive Bayes", "All", "All-Train", f1, acc, rec, prec]], columns =col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#test scores
test_predicted = clf.predict(X_test)
f1, acc, rec, prec = get_metrics(y_test, test_predicted)
results = results.append(pd.DataFrame([["Binary", "Naive Bayes", "All", "All-Test", f1, acc, rec, prec]], columns=col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#Experiment Scores
exp_predicted = clf.predict(X_exp)
f1, acc, rec, prec = get_metrics(y_exp, exp_predicted)
results = results.append(pd.DataFrame([["Binary", "Naive Bayes", "All", "Elec-Test", f1, acc, rec, prec]], columns=col_list))
print(str('{:04.2f}'.format(f1*100))+'%')

results.to_csv(output_dir, index=False)
total_results = total_results.append(results)


87.11%
73.53%
78.55%


In [ ]:
results

,Type,Model,Train Set,Test Set,F1,Accuracy,Recall,Precision
0,Binary,Naive Bayes,All,All-Train,0.871115,0.871149,0.871158,0.871561
0,Binary,Naive Bayes,All,All-Test,0.735283,0.865468,0.869572,0.692971
0,Binary,Naive Bayes,All,Elec-Test,0.785523,0.786175,0.816531,0.804185


Elec Bin

In [ ]:
output_dir ='/content/drive/MyDrive/cis519_final/Baselines/Naive Bayes/results_Elec_Bin.csv'
df_train = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/electronics_train_binaryposneg.json')
df_test = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/electronics_test_binaryposneg.json')
df_exp = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_test_binaryposneg.json')
df_train = pd.DataFrame().from_dict(df_train)
df_test = pd.DataFrame().from_dict(df_test)
df_exp = pd.DataFrame().from_dict(df_exp)

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
tf_idf = TfidfTransformer()
train_doc_matrix = cv.fit_transform(df_train['title_plus_review'])
X_train = tf_idf.fit_transform(train_doc_matrix)

test_doc_matrix = cv.transform(df_test['title_plus_review'])
X_test = tf_idf.transform(test_doc_matrix)

exp_doc_matrix = cv.transform(df_exp['title_plus_review'])
X_exp = tf_idf.transform(exp_doc_matrix)


y_train = df_train['pos_neg']
y_test = df_test['pos_neg']
y_exp = df_exp['pos_neg']

from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(text_counts, df['overall'], test_size = 0.25, random_state = 5)



clf = svm.SVC()
clf.fit(X_train, y_train)

On line0, 0.0 % done
On line5000, 0.002960002083841467 % done
On line10000, 0.005920004167682934 % done
On line15000, 0.008880006251524401 % done
On line20000, 0.011840008335365868 % done
On line25000, 0.014800010419207334 % done
On line30000, 0.017760012503048803 % done
On line35000, 0.02072001458689027 % done
On line40000, 0.023680016670731736 % done
On line45000, 0.026640018754573204 % done
On line50000, 0.02960002083841467 % done
On line55000, 0.03256002292225614 % done
On line60000, 0.035520025006097605 % done
On line65000, 0.03848002708993907 % done
On line70000, 0.04144002917378054 % done
On line75000, 0.04440003125762201 % done
On line80000, 0.04736003334146347 % done
On line85000, 0.050320035425304936 % done
On line90000, 0.05328003750914641 % done
On line95000, 0.05624003959298787 % done
On line100000, 0.05920004167682934 % done
On line105000, 0.06216004376067081 % done
On line110000, 0.06512004584451228 % done
On line115000, 0.06808004792835375 % done
On line120000, 0.071040

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
col_list = ['Type', "Model", "Train Set", 'Test Set', 'F1', 'Accuracy', 'Recall', 'Precision']
results = pd.DataFrame(columns=col_list)
#train scores
train_predicted = clf.predict(X_train)
f1, acc, rec, prec = get_metrics(y_train, train_predicted)
results = results.append(pd.DataFrame([["Binary", "Naive Bayes", "Elec", "Elec-Train", f1, acc, rec, prec]], columns =col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#test scores
test_predicted = clf.predict(X_test)
f1, acc, rec, prec = get_metrics(y_test, test_predicted)
results = results.append(pd.DataFrame([["Binary", "Naive Bayes", "Elec", "Elec-Test", f1, acc, rec, prec]], columns=col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#Experiment Scores
exp_predicted = clf.predict(X_exp)
f1, acc, rec, prec = get_metrics(y_exp, exp_predicted)
results = results.append(pd.DataFrame([["Binary", "Naive Bayes", "Elec", "ALL-Test", f1, acc, rec, prec]], columns=col_list))
print(str('{:04.2f}'.format(f1*100))+'%')

results.to_csv(output_dir, index=False)
total_results = total_results.append(results)


87.88%
86.08%
71.86%


In [ ]:
results

,Type,Model,Train Set,Test Set,F1,Accuracy,Recall,Precision
0,Binary,Naive Bayes,Elec,Elec-Train,0.878787,0.878789,0.878814,0.878856
0,Binary,Naive Bayes,Elec,Elec-Test,0.860762,0.865275,0.869770,0.856368
0,Binary,Naive Bayes,Elec,ALL-Test,0.718558,0.861104,0.831913,0.680219


In [ ]:
total_results.to_csv('/content/drive/MyDrive/cis519_final/Baselines/Naive Bayes/total_results.csv')

In [ ]:
total_results

,Type,Model,Train Set,Test Set,F1,Accuracy,Recall,Precision
0,5Cat,Naive Bayes,All,All-Train,0.581275,0.583144,0.583196,0.581655
0,5Cat,Naive Bayes,All,All-Test,0.404942,0.529363,0.493736,0.385027
0,5Cat,Naive Bayes,All,Elec-Test,0.430789,0.450334,0.456689,0.454546
0,5Cat,Naive Bayes,Elec,Elec-Train,0.570596,0.571873,0.572109,0.570359
0,5Cat,Naive Bayes,Elec,Elec-Test,0.495058,0.503864,0.501400,0.496625
0,5Cat,Naive Bayes,Elec,ALL-Test,0.377504,0.511363,0.444680,0.364700
0,Binary,Naive Bayes,All,All-Train,0.871115,0.871149,0.871158,0.871561
0,Binary,Naive Bayes,All,All-Test,0.735283,0.865468,0.869572,0.692971
0,Binary,Naive Bayes,All,Elec-Test,0.785523,0.786175,0.816531,0.804185
0,Binary,Naive Bayes,Elec,Elec-Train,0.878787,0.878789,0.878814,0.878856


Neutrality

In [ ]:
output_dir ='/content/drive/MyDrive/cis519_final/Baselines/Naive Bayes/results_All_neut.csv'
# df_train = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_train.json')
# df_test = fast_parse('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_test.json')
# df_experiment = pd.from_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/elec_test_5class.csv')
# df_train = pd.DataFrame().from_dict(df_train)
# df_test = pd.DataFrame().from_dict(df_test)

df_train = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_neutrality_binary.csv')
df_test = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_neutrality_binary_test.csv')
df_exp = pd.read_csv('/content/drive/Shareddrives/519 Project/Data/preprocessed/Final Data/all_neutrality_binary_test.csv')

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

token = RegexpTokenizer(r'[a-zA-Z0-9]+')

cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
tf_idf = TfidfTransformer()
train_doc_matrix = cv.fit_transform(df_train['title_plus_review'])
X_train = tf_idf.fit_transform(train_doc_matrix)

test_doc_matrix = cv.transform(df_test['title_plus_review'])
X_test = tf_idf.transform(test_doc_matrix)

exp_doc_matrix = cv.transform(df_exp['title_plus_review'])
X_exp = tf_idf.transform(exp_doc_matrix)


y_train = df_train['neutrality']
y_test = df_test['neutrality']
y_exp = df_exp['neutrality']

from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(text_counts, df['overall'], test_size = 0.25, random_state = 5)



clf = svm.SVC()
clf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
df_train.head()

,Unnamed: 0,Unnamed: 0.1,reviewText,summary,title_plus_review,pos_neg,neutrality,num_words,nw
0,0,23099,Lasted for about three cycles and won't keep a...,Lasted for about three cycles and won't keep a...,lasted for about three cycles and will not kee...,0,1,41,30
1,1,167477,Amazing quality for Dog apparel! However the s...,Super disappointed about size runs extra small,super disappointed about size runs extra small...,0,1,49,43
2,2,205965,Not very durable :(,One Star,one star not very durable,0,1,5,3
3,3,47624,Really really poor quality- gave off BLACK DYE...,Covered in black dye,covered in black dye really really poor qualit...,0,1,16,12
4,4,185644,"Go with the other cheaper lincoln log sets, th...",Flimsy can,flimsy can go with the other cheaper lincoln l...,0,1,47,45


In [ ]:
results = pd.DataFrame(columns=col_list)
#train scores
train_predicted = clf.predict(X_train)
f1, acc, rec, prec = get_metrics(y_train, train_predicted)
results = results.append(pd.DataFrame([["Neutrality", "Naive Bayes", "All", "All-Train", f1, acc, rec, prec]], columns =col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#test scores
test_predicted = clf.predict(X_test)
f1, acc, rec, prec = get_metrics(y_test, test_predicted)
results = results.append(pd.DataFrame([["Neutrality", "Naive Bayes", "All", "All-Test", f1, acc, rec, prec]], columns=col_list))
print(str('{:04.2f}'.format(f1*100))+'%')
#Experiment Scores

results.to_csv(output_dir, index=False)
total_results = total_results.append(results)

78.52%
55.23%


In [ ]:
results

,Type,Model,Train Set,Test Set,F1,Accuracy,Recall,Precision
0,Neutrality,Naive Bayes,All,All-Train,0.785196,0.785263,0.785264,0.785621
0,Neutrality,Naive Bayes,All,All-Test,0.552255,0.677471,0.713086,0.580629
